# Trotsky

<https://thetvdb.com/series/trotsky>

![Trotsky Banner](https://artworks.thetvdb.com/banners/graphical/5c8de44196ea4.jpg)


These notes preserve my memories of auto-generating [Kodi](https://kodi.tv/) [[GitHub](https://github.com/xbmc)] `tvshow` and `episodedetails` XML files in `*.nfo` formats.

This `*.nfo` format is kind of covered in the following documents:

- [NFO files](https://kodi.wiki/view/NFO_files)
- [NFO files/TV shows](https://kodi.wiki/view/NFO_files/TV_shows)
- [NFO files/Episodes](https://kodi.wiki/view/NFO_files/Episodes)


In [1]:
from IPython.core.display import display, HTML

import songhay.utilities.elementtree as ET
import songhay.utilities.soup as bs

import songhay.soup.thetvdb.series as series

## generating `tvshow` data


In [2]:
from tokenize import String


series_location = 'https://thetvdb.com/series/trotsky'
series_soup = bs.getSoup(series_location)
series_basic_info = dict(series.yieldSeriesBasicInfo(series_soup))

series_data = dict(
    uniqueids=[
        {
            'type': 'tvdb',
            'uniqueid': 'series/trotsky'
        },
        {
            'type': 'imdb',
            'uniqueid': 'tt7599132'
        },
    ],
    title=series.getSeriesTitle(series_soup),
    plot=series.getSeriesPlot(series_soup),
    thumbs=[
        {
            'aspect': 'banner',
            'dim': None,
            'season': None,
            'src': 'https://artworks.thetvdb.com/banners/graphical/5c8de44196ea4.jpg'
        },
        {
            'aspect': 'poster',
            'dim': None,
            'season': None, # maps to the top-level movie poster
            'src': 'https://artworks.thetvdb.com/banners/posters/5b510d2263ec0.jpg'
        },
    ],
    genres=series_basic_info['Genres'],
    actors=list(series.yieldSeriesActors(series_soup))
)

ET.getTVShowXmlTree(series_data).write(
    './beautifulsoup-trotsky/tvshow.nfo',
    encoding='utf-8',
    xml_declaration=True)

## generating `episodedetails` data

This title does not have episode data on `thetvdb.com` so we turn to `imdb.com`:

In [24]:
episode_location = 'https://www.imdb.com/title/tt7599132/episodes?ref_=tt_eps_sm'

episode_soup = bs.getSoup(episode_location)

def yieldImdbEpisodeData(soup):
    div = soup.select_one('div.list.detail.eplist')
    div_items = div.find_all('div', { 'class': 'list_item' })
    for item in div_items:
        div_info = item.select_one('div.info')

        episode_number = div_info.select_one('meta')['content']
        episode = f'S01E0{episode_number}'
        title = div_info.strong.a.string
        year = div_info.select_one('div.airdate').string.strip().split(' ')[-1]
        plot = div_info.select_one('div.item_description').string.strip()

        yield {
            'episode' : episode,
            'title': title,
            'plot': plot,
            'year': year
        }

episode_data = list(yieldImdbEpisodeData(episode_soup))
episode_data

[{'episode': 'S01E01',
  'title': 'Episode #1.1',
  'plot': '1940, Mexico. Leon Trotsky, one of the leaders of the Russian revolution, is certain that Stalin will try to assassinate him. He decides to share his story with a young journalist, starting with how he adopted the surname Trotsky.',
  'year': '2017'},
 {'episode': 'S01E02',
  'title': 'Episode #1.2',
  'plot': "It's 1903. Trotsky wants to lead the revolution, but to become a leader he needs to defeat a number of opponents, including Vladimir Lenin. At the same time he gets involved with Natalya, a woman from a completely different social class.",
  'year': '2017'},
 {'episode': 'S01E03',
  'title': 'Episode #1.3',
  'plot': "It's 1905. The working class is angry and the revolution seems imminent. Despite the general eagerness to fight, Trotsky believes it's too soon for the revolution to be successful.",
  'year': '2017'},
 {'episode': 'S01E04',
  'title': 'Episode #1.4',
  'plot': "It's 1917. After spending a number of years

In [25]:
nfo_location_template = f"./beautifulsoup-trotsky/Trotsky"

In [26]:
for item in episode_data:
    episode = item['episode']
    year = item['year']
    xml_tree = ET.getEpisodeDetailsXmlTree(item['title'], item['plot'])
    ET.writeEpisodeDetailsXml(nfo_location_template, episode, year, xml_tree)


## subtitles

[Subtitles](https://www.opensubtitles.org/en/ssearch/sublanguageid-all/idmovie-699887) for this title were available at `opensubtitles.org`.

@[BryanWilhite](https://twitter.com/BryanWilhite)
